In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("data.csv")
batact = pd.DataFrame()
fin_batact = pd.DataFrame()

In [ ]:
df["COMBINED_DT"] = pd.to_datetime(df["DATE"]+' '+df["TIME"])
df["DATE"] = pd.to_datetime(df["DATE"],format="%m/%d/%y")

/tmp/ipykernel_6532/2760743003.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["COMBINED_DT"] = pd.to_datetime(df["DATE"]+' '+df["TIME"])


In [ ]:
df.head(5)

,DATE,TIME,STATUS,NAME,TYPE,SOURCE,PRESENT,TECHNOLOGY,CYCLE_COUNT,VOLTAGE_MIN_DESIGN,...,CURRENT_NOW,CHARGE_FULL_DESIGN,CHARGE_FULL,CHARGE_NOW,CAPACITY,CAPACITY_LEVEL,MODEL_NAME,MANUFACTURER,SERIAL_NUMBER,COMBINED_DT
0,2023-07-30,22:57:13,Active,BAT0,Battery,Charging,1,Li-ion,0,11400000,...,13000,3684000,1814000,610000,33,Normal,DELL-FDRHM08,LGC-LGC3.65,34838.0,2023-07-30 22:57:13
1,2023-07-30,22:53:45,Active,BAT0,Battery,Discharging,1,Li-ion,0,11400000,...,174000,3684000,1814000,610000,33,Normal,DELL-FDRHM08,LGC-LGC3.65,34838.0,2023-07-30 22:53:45
2,2023-07-30,22:08:34,Active,BAT0,Battery,Discharging,1,Li-ion,0,11400000,...,12000,3684000,1814000,631000,34,Normal,DELL-FDRHM08,LGC-LGC3.65,34838.0,2023-07-30 22:08:34
3,2023-07-30,22:08:21,Active,BAT0,Battery,Discharging,1,Li-ion,0,11400000,...,3684000,1814000,630000,34,Normal,DELL-FDRHM08,LGC-LGC3.65,34838,NaN,2023-07-30 22:08:21
4,2023-07-30,21:52:34,Active,BAT0,Battery,Charging,1,Li-ion,0,11400000,...,1051000,3684000,1814000,358000,19,Normal,DELL-FDRHM08,LGC-LGC3.65,34838.0,2023-07-30 21:52:34


In [ ]:
curr_date = df["DATE"].iloc[-1]
last_date = df["DATE"][0]

In [ ]:
rev_df = df[::-1].reset_index(drop=True)

In [ ]:
sus_loc = rev_df.loc[rev_df["STATUS"]=="Suspended"]
act_ind = sus_loc.index+1

In [ ]:
act_loc = rev_df.loc[act_ind]

In [ ]:
sus_loc = sus_loc.reset_index(drop=True)
act_loc = act_loc.reset_index(drop=True)

In [ ]:
def format_timedelta(timedelta_object):
    days = timedelta_object.days
    seconds = timedelta_object.seconds
    hours, remainder = divmod(seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{24 * days + hours:02d}:{minutes:02d}:{seconds:02d}"


In [ ]:
temp_act = pd.DataFrame()
if rev_df["STATUS"][0] == "Active":
    temp_act["DATE"] = pd.Series(rev_df["DATE"][0])
    temp_act["TIMESTAMP"] = pd.Series(pd.to_datetime(sus_loc["COMBINED_DT"][0]) - pd.to_datetime(rev_df["COMBINED_DT"][0]))
    batact["DATE"] = act_loc["DATE"]
    batact["TIMESTAMP"] = pd.to_datetime(act_loc["COMBINED_DT"]) - pd.to_datetime(sus_loc["COMBINED_DT"])
    batact = pd.concat([temp_act,batact],ignore_index=True)

In [ ]:
if rev_df["STATUS"][0] == "Suspended":
    batact["DATE"] = act_loc["DATE"]
    batact["TIMESTAMP"] = pd.to_datetime(act_loc["COMBINED_DT"]) - pd.to_datetime(sus_loc["COMBINED_DT"])

In [ ]:
batact

,DATE,TIMESTAMP
0,2023-05-03,0 days 00:00:34
1,2023-05-03,0 days 00:00:50
2,2023-05-03,0 days 06:54:16
3,2023-05-03,0 days 00:02:13
4,2023-05-03,0 days 00:00:38
...,...,...
209,2023-07-30,0 days 08:00:33
210,2023-07-30,0 days 00:16:36
211,2023-07-30,0 days 00:01:16
212,2023-07-30,0 days 01:18:50


In [ ]:
curr_date=pd.to_datetime(df["DATE"].iloc[-1])
week_d=curr_date+pd.DateOffset(weeks=1)
st_l = [];en_l = []; ch_l = []

In [ ]:
while True:
    st_l.append(curr_date)
    en_l.append(week_d)
    ch_l.append(batact.loc[(batact["DATE"]>=curr_date)&(batact["DATE"]<=week_d)]["TIMESTAMP"].sum())
    curr_date = curr_date+pd.DateOffset(weeks=1)
    week_d = week_d+pd.DateOffset(weeks=1)
    if(week_d>=batact["DATE"].iloc[-1]+pd.DateOffset(days=1)):
        break

In [ ]:
if curr_date<batact["DATE"].iloc[-1]:
    week_d = batact["DATE"].iloc[-1]
    st_l.append(curr_date)
    en_l.append(week_d)
    ch_l.append(batact.loc[(batact["DATE"]>=curr_date)&(batact["DATE"]<=week_d)]["TIMESTAMP"].sum())

In [ ]:
fin_batact["START_DATE"] = pd.Series(st_l)
fin_batact["END_DATE"] = pd.Series(en_l)
fin_batact["TIME_USED"] = pd.Series(ch_l).apply(format_timedelta)

In [ ]:
fin_batact

,START_DATE,END_DATE,TIME_USED
0,2023-05-03,2023-05-10,92:17:35
1,2023-05-10,2023-05-17,106:30:06
2,2023-05-17,2023-05-24,68:38:25
3,2023-05-24,2023-05-31,60:04:08
4,2023-05-31,2023-06-07,90:16:30
5,2023-06-07,2023-06-14,142:44:21
6,2023-06-14,2023-06-21,94:28:58
7,2023-06-21,2023-06-28,129:43:19
8,2023-06-28,2023-07-05,92:13:00
9,2023-07-05,2023-07-12,113:58:38


## Battery Usage Activity - part 2

In [1]:
import pandas as pd

In [2]:
batuse = pd.read_csv("batusageact.csv")

In [3]:
batuse.head(5)

,DATETIME,STATE,STATUS,GROUP
0,2025-02-19 20:51:46,Active,Charging,1.0
1,2025-02-19 18:20:56,Active,Discharging,1.0
2,2025-02-19 16:55:21,Active,Charging,1.0
3,2025-02-19 16:43:10,Active,Discharging,1.0
4,2025-02-19 16:37:29,Active,Charging,1.0


In [4]:
batuse["DATETIME"] = pd.to_datetime(batuse["DATETIME"], format="%Y-%m-%d %H:%M:%S")

In [5]:
batuse_ch = batuse[batuse["STATUS"]=="Charging"]
batuse_dc = batuse[batuse["STATUS"]!="Charging"]

In [6]:
batuse['DIFFERENCE'] = batuse["DATETIME"] - batuse.shift(-1)["DATETIME"]
batuse['DIFFERENCE'] = batuse['DIFFERENCE'].fillna(pd.Timedelta(seconds=0))

In [7]:
datetime_df = pd.DataFrame()

In [8]:
datetime_df["START_DATE"] = batuse.groupby("GROUP").agg({"DATETIME":"first"})
datetime_df["END_TIME"] = batuse.groupby("GROUP").agg({"DATETIME":"last"})
datetime_df.reset_index(drop=True,inplace=True)

In [9]:
datetime_df

,START_DATE,END_TIME
0,2025-02-19 20:51:46,2025-02-15 09:50:05
1,2025-02-13 20:42:47,2025-02-13 09:39:49
2,2025-02-12 21:35:50,2025-02-12 08:18:16
3,2025-02-11 18:31:38,2025-02-06 17:25:07
4,2025-02-05 14:56:42,2025-01-30 08:12:51
5,2025-01-29 21:50:26,2025-01-18 10:20:13
6,2025-01-17 12:15:04,2025-01-08 14:22:25
7,2025-01-08 14:01:23,2025-01-06 16:55:00
8,2025-01-06 14:29:31,2024-12-26 18:23:01
9,2024-12-26 17:56:23,2024-12-22 12:07:32


In [10]:
batuse_ch = batuse[batuse["STATUS"]=="Charging"]
batuse_dc = batuse[batuse["STATUS"]!="Charging"]

In [11]:
batuse_ch_act = batuse_ch[batuse_ch["STATE"]=="Active"]
batuse_ch_low = batuse_ch[batuse_ch["STATE"]=="Low Power"]

In [12]:
batuse_dc_act = batuse_dc[batuse_dc["STATE"]=="Active"]
batuse_dc_low = batuse_dc[batuse_dc["STATE"]=="Low Power"]

In [13]:
batuse_ch_act_gr = batuse_ch_act.groupby("GROUP")["DIFFERENCE"].sum().reset_index()
batuse_ch_low_gr = batuse_ch_low.groupby("GROUP")["DIFFERENCE"].sum().reset_index()

In [14]:
batuse_dc_act_gr = batuse_dc_act.groupby("GROUP")["DIFFERENCE"].sum().reset_index()
batuse_dc_low_gr = batuse_dc_low.groupby("GROUP")["DIFFERENCE"].sum().reset_index()

In [15]:
batuse_ch_act_gr = batuse_ch_act_gr.rename(columns={"DIFFERENCE":"CHARGING_ACTIVE"})
batuse_ch_low_gr = batuse_ch_low_gr.rename(columns={"DIFFERENCE":"CHARGING_LOW_POWER"})
batuse_dc_act_gr = batuse_dc_act_gr.rename(columns={"DIFFERENCE":"BATTERY_ACTIVE"})
batuse_dc_low_gr = batuse_dc_low_gr.rename(columns={"DIFFERENCE":"BATTERY_LOW_POWER"})

In [16]:
df = batuse_dc_act_gr.merge(batuse_dc_low_gr, on="GROUP", how="outer")\
            .merge(batuse_ch_act_gr, on="GROUP", how="outer")\
            .merge(batuse_ch_low_gr, on="GROUP", how="outer")

In [22]:
final_df = datetime_df.merge(df, on=datetime_df.index, how="outer")

In [18]:
final_df = final_df.fillna("-")

In [24]:
final_df = final_df.drop(columns=["key_0","GROUP"])

In [25]:
final_df

,START_DATE,END_TIME,BATTERY_ACTIVE,BATTERY_LOW_POWER,CHARGING_ACTIVE,CHARGING_LOW_POWER
0,2025-02-19 20:51:46,2025-02-15 09:50:05,1 days 07:23:27,NaT,1 days 05:05:46,NaT
1,2025-02-13 20:42:47,2025-02-13 09:39:49,0 days 06:23:05,NaT,0 days 04:41:12,NaT
2,2025-02-12 21:35:50,2025-02-12 08:18:16,0 days 10:41:01,NaT,0 days 09:52:03,NaT
3,2025-02-11 18:31:38,2025-02-06 17:25:07,2 days 08:29:58,NaT,2 days 23:30:12,NaT
4,2025-02-05 14:56:42,2025-01-30 08:12:51,2 days 18:17:20,NaT,3 days 01:09:01,NaT
5,2025-01-29 21:50:26,2025-01-18 10:20:13,2 days 18:16:20,NaT,4 days 16:00:38,NaT
6,2025-01-17 12:15:04,2025-01-08 14:22:25,3 days 06:55:33,NaT,3 days 20:06:48,NaT
7,2025-01-08 14:01:23,2025-01-06 16:55:00,1 days 00:25:34,NaT,0 days 11:06:50,NaT
8,2025-01-06 14:29:31,2024-12-26 18:23:01,3 days 14:13:46,NaT,6 days 16:22:26,NaT
9,2024-12-26 17:56:23,2024-12-22 12:07:32,2 days 03:53:55,NaT,2 days 07:59:42,NaT


In [ ]:
table_header = """
<table border="1">
        <thead>
            <tr class="header-row">
                <th rowspan="2">Start Date</th>
                <th rowspan="2">End Date</th>
                <th colspan="2">On Battery</th>
                <th colspan="2">While Charging</th>
            </tr>
            <tr>
                <th>Active</th>
                <th>Low Power</th>
                <th>Active</th>
                <th>Low Power</th>
            </tr>
        </thead>
"""

In [20]:
table_body = ""

In [21]:
for i in range(len(final_df)):
    table_body += f"""
            <tr>
                <td>{datetime_df["START_DATE"][i]}</td>
                <td>{datetime_df["END_TIME"][i]}</td>
                <td>{final_df["BATTERY_ACTIVE"][i]}</td>
                <td>{final_df["BATTERY_LOW_POWER"][i]}</td>
                <td>{final_df["CHARGING_ACTIVE"][i]}</td>
                <td>{final_df["CHARGING_LOW_POWER"][i]}</td>
            </tr>
    """

In [22]:
table_body

'\n            <tr>\n                <td>2025-02-19 20:51:46</td>\n                <td>2025-02-15 09:50:05</td>\n                <td>1 days 07:23:27</td>\n                <td>-</td>\n                <td>1 days 05:05:46</td>\n                <td>-</td>\n            </tr>\n    \n            <tr>\n                <td>2025-02-13 20:42:47</td>\n                <td>2025-02-13 09:39:49</td>\n                <td>0 days 06:23:05</td>\n                <td>-</td>\n                <td>0 days 04:41:12</td>\n                <td>-</td>\n            </tr>\n    \n            <tr>\n                <td>2025-02-12 21:35:50</td>\n                <td>2025-02-12 08:18:16</td>\n                <td>0 days 10:41:01</td>\n                <td>-</td>\n                <td>0 days 09:52:03</td>\n                <td>-</td>\n            </tr>\n    \n            <tr>\n                <td>2025-02-11 18:31:38</td>\n                <td>2025-02-06 17:25:07</td>\n                <td>2 days 08:29:58</td>\n                <td>

In [23]:
table_footer = """ 
    </tbody>
</table>
"""

In [24]:
full_table = table_header + table_body + table_footer

In [25]:
with open("battery_activity.html", "w") as f:
    f.write(full_table)